## Data Exploration
## Content
  * Load data 
  * Investigate SalePrice
  * Remove bad fields
  * Dealing with missing data
  * Correlation matrix for numerical features
  * Encode categorical features to numerical
  * Correlation matrix for encoded categorical features
  * Investigate Logarifm transformations
  * Investigate Pow transformations

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats
from pymongo import MongoClient
import warnings
%matplotlib inline

warnings.filterwarnings('ignore')

In [ ]:
MONGODB_URL = os.environ['AMES_MONGODB_URI']
client = MongoClient(MONGODB_URL)
db = client.get_default_database()

In [ ]:
data = db["train_data"].find({})
df_train = pd.DataFrame(list(data))
df_train.drop(columns=["_id"], inplace=True)
df_train.shape

In [ ]:
data = db["test_data"].find({})
df_test = pd.DataFrame(list(data))
df_test.drop(columns=["_id"], inplace=True)
df_test.shape

In [ ]:
full_frame = pd.concat([df_train, df_test])
full_frame.shape

## Investigate SalePrice

In [ ]:
#histogram
sns.set(rc={'figure.figsize':(15,5)})
fig, ax = plt.subplots(1,2)
sns.distplot(np.log1p(full_frame['SalePrice']), fit=norm, ax=ax[0])
sns.distplot(full_frame['SalePrice'], fit=norm, ax=ax[1])
ax[0].set_title("SalePrice: after logarithm transform")
ax[1].set_title("SalePrice: original")
fig.show()

#normal probability plot
fig, ax = plt.subplots(1,2)
stats.probplot(np.log1p(full_frame['SalePrice']), plot=ax[0])
stats.probplot(full_frame['SalePrice'], plot=ax[1])
ax[0].set_title("SalePrice: after logarithm transform")
ax[1].set_title("SalePrice: original")
fig.show()

You should choose True value for applyLogTransformForSalePrice if after logarithm transform SalePrice became more corresponded with normal distribution

In [ ]:
applyLogTransformForSalePrice = True

In [ ]:
if applyLogTransformForSalePrice:
    full_frame['SalePrice'] = np.log1p(full_frame['SalePrice'])

### Remove bad fields

In [ ]:
features = set(full_frame.columns.tolist())
features.remove('SalePrice')

Choose fields that cannot be taken for training for some reasons

In [ ]:
# features.remove('OverallQual')

In [ ]:
len(features)

### Dealing with missing data

In [ ]:
#missing data
total = full_frame.isnull().sum().sort_values(ascending=False)
percent = (full_frame.isnull().sum()/full_frame.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(5)

In [ ]:
# remove features where data is missed more then 5 times
features = features.difference(set((missing_data[missing_data['Total'] > 5]).index.tolist()))

In [ ]:
# remove left images where data is missed
full_frame = full_frame.drop(full_frame.loc[full_frame.isnull().any(axis=1)].index)
full_frame[list(features)].isnull().sum().max() #just checking that there's no missing data missing...

In [ ]:
# features left
len(features)

### Lists of numerical and categorical features

In [ ]:
# get categorical features list
g = {k.name: v for k, v in full_frame[list(features)].columns.to_series().groupby(full_frame.dtypes).groups.items()}
categorical = g["object"].tolist()
numerical = g["float64"].tolist() + g["int64"].tolist()
g

### Correlation matrix for numerical features

In [ ]:
# numerical features list
numerical

In [ ]:
len(numerical)

In [ ]:
numerical_df = full_frame[numerical + ['SalePrice']].copy(True)

#correlation matrix
corrmat = numerical_df.corr().abs()

# sort columns in df by SalePrice correlation
cols = corrmat.sort_values(["SalePrice"])['SalePrice'].index
# get new correlation matrix with sorted columns
cm = numerical_df[cols].corr().abs()
# show plot
sns.set(rc={'figure.figsize':(10,10)})
sns.heatmap(cm, vmax=.8, square=True, yticklabels=cols.values, xticklabels=cols.values)

In [ ]:
# dealing with features which highly correlated with each other
# features that correlated with each other more then this threshold should be considered to remove
# Please enter threshold
thresh = 0.7

In [ ]:
# mask to go through only under diagonal triangle
mask = np.ones(cm.shape,dtype='bool')
mask[np.triu_indices(len(cm))] = False

highly_corr_cm = cm.drop(index=['SalePrice'],columns=['SalePrice'])[(cm>thresh)&mask].dropna(axis=0, how='all').dropna(axis=1, how='all')
highly_corr_cm = highly_corr_cm.append(cm['SalePrice'][highly_corr_cm.columns])
highly_corr_cm['SalePrice'] = cm['SalePrice'][highly_corr_cm.index]
highly_corr_cm

In [ ]:
# Please enter features that should be removed according to table above 
columns_to_remove = []

In [ ]:
numerical_df.drop(columns=columns_to_remove, inplace=True)

Define attributes with highest correlation and transformation type which should be applied for it. Please enter wished correlation threshold for numerical values:

In [ ]:
corr_threshold_for_num_features = 0.3

In [ ]:
numerical_df_log = numerical_df.copy(True)
numerical_df_log.drop(columns=['SalePrice'], inplace=True)
for feature in numerical_df_log.columns:
    numerical_df_log[feature] = numerical_df_log[feature].apply(lambda x: 0 if x <= 1 else np.log(x))
numerical_df_log["SalePrice"] = numerical_df["SalePrice"]
corrmat_log = numerical_df_log.corr().abs()


numerical_df_pow = numerical_df.copy(True)
numerical_df_pow = numerical_df_pow.drop(columns=['SalePrice']).apply(lambda x: x**2)
numerical_df_pow["SalePrice"] = numerical_df["SalePrice"]
corrmat_pow = numerical_df_pow.corr().abs()

magic_num_table = pd.DataFrame()
magic_num_table["log"] = corrmat_log["SalePrice"]
magic_num_table["pow"] = corrmat_pow["SalePrice"]
magic_num_table["origin"] = corrmat["SalePrice"]
magic_num_table["highest_value"] = magic_num_table.max(axis=1)

magic_num_table["transform_type"] = "magic_num_table"
magic_num_table["transform_type"][magic_num_table["log"] == magic_num_table["highest_value"]] = "log"
magic_num_table["transform_type"][magic_num_table["pow"] == magic_num_table["highest_value"]] = "pow"
magic_num_table["transform_type"][magic_num_table["origin"] == magic_num_table["highest_value"]] = "origin"

magic_num_table.drop(index=["SalePrice"], inplace=True)

magic_num_table = magic_num_table.sort_values('origin', ascending=False)
magic_num_table = magic_num_table[magic_num_table["origin"] > corr_threshold_for_num_features]
magic_num_table

selected numerical features

In [ ]:
magic_num_table["transform_type"]

### Encode categorical features to numerical

In [ ]:
# categorical features list
categorical

In [ ]:
len(categorical)

In [ ]:
# perform ANOVA test 

def anova(frame, qualitative):
    anv = pd.DataFrame()
    anv['feature'] = qualitative
    pvals = []
    for c in qualitative:
        samples = []
        for cls in frame[c].unique():
            s = frame[frame[c] == cls]['SalePrice'].values
            samples.append(s.astype(np.float64))
        pval = stats.f_oneway(*samples)[1]
        pvals.append(pval)
    anv['pval'] = pvals
    return anv.sort_values('pval')

a = anova(full_frame.copy(True), g["object"].tolist())
a['disparity'] = np.log(1./a['pval'].values)
sns.set(rc={'figure.figsize':(15,10)})
sns.barplot(data=a, y='feature', x='disparity')

In [ ]:
# perform encoding

dummies = {}

def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()
    
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature] = o
    
    return ordering
    
for c in categorical:  
    dummies[c] = encode(full_frame, c)

In [ ]:
categorical_df = full_frame[categorical + ['SalePrice']]
categorical_df['SalePrice'] = np.log1p(categorical_df['SalePrice'])

#correlation matrix
corrmat = categorical_df.corr().abs()
cols = corrmat.sort_values(["SalePrice"])['SalePrice'].index
cm = categorical_df[cols].corr().abs()
sns.set(rc={'figure.figsize':(10,10)})
sns.heatmap(cm, vmax=.8, square=True, yticklabels=cols.values, xticklabels=cols.values)

In [ ]:
mask = np.ones(cm.shape,dtype='bool')
mask[np.triu_indices(len(cm))] = False

In [ ]:
# dealing with features which highly correlated with each other
# features that correlated with each other more then this threshold should be considered to remove
# Please enter threshold
thresh = 0.7

In [ ]:
highly_corr_cm = cm.drop(index=['SalePrice'],columns=['SalePrice'])[(cm>thresh)&mask].dropna(axis=0, how='all').dropna(axis=1, how='all')
highly_corr_cm = highly_corr_cm.append(cm['SalePrice'][highly_corr_cm.columns])
highly_corr_cm['SalePrice'] = cm['SalePrice'][highly_corr_cm.index]
highly_corr_cm

In [ ]:
# Please enter features that should be removed according to table above 
columns_to_remove = []

In [ ]:
categorical_df.drop(columns=columns_to_remove, inplace=True)

Define attributes with highest correlation and transformation type which should be applied for it. Please enter wished correlation threshold for categorical values:

In [ ]:
corr_threshold_for_cat_features = 0.4

In [ ]:
categorical_df_log = categorical_df.copy(True)
categorical_df_log = categorical_df_log.drop(columns=['SalePrice']).apply(lambda x: np.log1p(x))
categorical_df_log["SalePrice"] = categorical_df["SalePrice"]
corrmat_log = categorical_df_log.corr().abs()

categorical_df_pow = categorical_df.copy(True)
categorical_df_pow = categorical_df_pow.drop(columns=['SalePrice']).apply(lambda x: x**2)
categorical_df_pow["SalePrice"] = categorical_df["SalePrice"]
corrmat_pow = categorical_df_pow.corr().abs()

magic_cat_table = pd.DataFrame()
magic_cat_table["log"] = corrmat_log["SalePrice"]
magic_cat_table["pow"] = corrmat_pow["SalePrice"]
magic_cat_table["origin"] = corrmat["SalePrice"]
magic_cat_table["highest_value"] = magic_cat_table.max(axis=1)

magic_cat_table["transform_type"] = "magic_cat_table"
magic_cat_table["transform_type"][magic_cat_table["log"] == magic_cat_table["highest_value"]] = "log"
magic_cat_table["transform_type"][magic_cat_table["pow"] == magic_cat_table["highest_value"]] = "pow"
magic_cat_table["transform_type"][magic_cat_table["origin"] == magic_cat_table["highest_value"]] = "origin"

magic_cat_table.drop(index=["SalePrice"], inplace=True)

magic_cat_table = magic_cat_table.sort_values('origin', ascending=False)
magic_cat_table = magic_cat_table[magic_cat_table["origin"] > corr_threshold_for_cat_features]
magic_cat_table

selected categorical features

In [ ]:
magic_cat_table["transform_type"]

### Summarize

In [ ]:
magic_table = pd.concat([magic_cat_table, magic_num_table]).sort_values('origin', ascending=False)
magic_table

## Transformation proofs

In [ ]:
if applyLogTransformForSalePrice:
    full_frame['SalePrice'] = np.expm1(full_frame['SalePrice'])
features = magic_table.index.tolist()

### Logarifm transformations

In [ ]:
logarifm_frame = full_frame[features + ['SalePrice']].copy(True)
for feature in logarifm_frame.columns:
    logarifm_frame[feature] = logarifm_frame[feature].apply(lambda x: 0 if x <= 1 else np.log(x))

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
for feature in features:
    plt.figure()
    fig, ax = plt.subplots(1,2)
    stats.probplot(logarifm_frame[feature], plot=ax[0])
    stats.probplot(full_frame[feature], plot=ax[1])
    ax[0].set_title(feature + ": after logarithm transform")
    ax[1].set_title(feature + ": original")
    plt.show()

### Pow transformations

In [ ]:
pow_frame = full_frame[features + ['SalePrice']].copy(True)
for feature in logarifm_frame.columns:
    pow_frame[feature] = pow_frame[feature].apply(lambda row: row**2)

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
for feature in features:
    plt.figure()
    fig, ax = plt.subplots(1,2)
    stats.probplot(pow_frame[feature], plot=ax[0])
    stats.probplot(full_frame[feature], plot=ax[1])
    ax[0].set_title(feature + ": after pow transform")
    ax[1].set_title(feature + ": original")
    plt.show()

### Save results for future learning

In [ ]:
notebook_collection = db["notebook"]
notebook_collection.remove({})
notebook_collection.insert_many(full_frame.to_dict('records'))